In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [9]:
from __future__ import print_function

In [10]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [11]:
header = ["color", "diameter", "label"]

In [12]:
def unique_vals(rows, col):
    
    return set([row[col] for row in rows])

In [13]:
unique_vals(training_data, 0)

{'Green', 'Red', 'Yellow'}

In [14]:
def class_counts(rows):
    
    counts = {}  
    for row in rows:
        
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [15]:
class_counts(training_data)

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [16]:
def is_numeric(value):
    
    return isinstance(value, int) or isinstance(value, float)

In [17]:
is_numeric(7)

True

In [18]:
class Question:
   

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [19]:
Question(1, 3)

Is diameter >= 3?

In [20]:
q = Question(0, 'Green')
q

Is color == Green?

In [21]:
example = training_data[0]

In [22]:
q.match(example)

True

In [23]:
def partition(rows, question):
    
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [24]:
true_rows, false_rows = partition(training_data, Question(0, 'Red'))


true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [25]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [26]:

def gini(rows):
   
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [27]:
no_mixing = [['Apple'],
              ['Apple']]

gini(no_mixing)

0.0

In [23]:
some_mixing = [['Apple'],
               ['Orange']]
gini(some_mixing)

0.5

In [24]:
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
gini(lots_of_mixing)

0.7999999999999998

In [56]:
def info_gain(left, right, current_uncertainty):
    
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [26]:
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [27]:
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [28]:
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [57]:
true_rows, false_rows = partition(training_data, Question(0,'Red'))


true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [58]:
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [59]:
true_rows, false_rows = partition(training_data, Question(0,'Green'))


true_rows

[['Green', 3, 'Apple']]

In [32]:
false_rows

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [61]:
def find_best_split(rows):
    
    best_gain = 0  # keep the best information gain
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  

    for col in range(n_features):  

        values = set([row[col] for row in rows])

        for val in values:  

            question = Question(col, val)

            # splitting the dataset
            true_rows, false_rows = partition(rows, question)

            
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [34]:
best_gain, best_question = find_best_split(training_data)
best_question

Is diameter >= 3?

In [35]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)


In [36]:

class Decision_Node:
    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch


In [62]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [63]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    #function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    #function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [64]:
my_tree = build_tree(training_data)

In [65]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [66]:
def classify(row, node):
   
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [67]:
classify(training_data[0], my_tree)

{'Apple': 1}

In [68]:
def print_leaf(counts):
    
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [44]:
print_leaf(classify(training_data[0], my_tree))

{'Apple': '100%'}

In [45]:
print_leaf(classify(training_data[1], my_tree))

{'Apple': '50%', 'Lemon': '50%'}

In [46]:
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [70]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
